In [ ]:
%load_ext autoreload
%autoreload 2

import edunn as nn
import numpy as np

from edunn import utils

# Inicialización de parámetros

Las capas `AddConstant` y `MultiplyConstant` creadas anteriormente no fueron definidas como _capas con parámetros_, sino que fueron definidas tomando un valor fijo provisto al momento de ser creadas.

Al definirla como una _capa con parámetros_, se pueden inicializar los parámetros de forma más flexible (ej: definiendo valores aleatorios) utilizando un objeto `Initializer`.

# Initializer
Un objeto `Initializer` permite delegar la responsabilidad de inicializar los parámetros de una capa. Se pueden definir distintas estrategias de inicialización, por ejemplo:

- Inicialización en un valor constante
- Inicialización en 0 (caso particular del anterior)
- Inicialización en valores aleatorios

# Creación e inicialización

Para esta guía crearemos una capa `DummyLayer` para comprender el uso del Initializer.

La capa `DummyLayer` tiene un vector de parámetros `c`, que debe crearse e inicializarse de alguna forma. Además, este parámetro se registra en la capa para poder ser accedido posteriormente.
La capa funciona para arrays que tengan el mismo tamaño que los parámetros.

Observa la implementación del método `__init__` de la capa `DummyLayer` para ver como el parámetro `c` es creado. Verás que se utiliza un `Initializer` para establecer su valor inicial.

Al crear la capa se le puede pasar un objeto de tipo `Initializer` que va a crear y asignarle el valor inicial al parámetro `c`. Por defecto, si no recibe ningún `Initializer`, el parámetro se inicializará con ceros utilizando la clase `initializers.Zero`


In [ ]:
from edunn.model import ModelWithParameters, ParameterSet
from edunn import initializers

class DummyLayer(ModelWithParameters):

    def __init__(self, output_size: int, initializer: initializers.Initializer = None, name=None):
        super().__init__(name=name)

        if initializer is None:
            initializer = initializers.Zero()

        c = initializer.create((output_size,))
        self.register_parameter('c', c)

    def forward(self, *x) -> np.ndarray:
        pass

    def backward(self, dE_dy: np.ndarray) -> (np.ndarray, ParameterSet):
        pass



Examinando la implementación de la clase `Initializers.Zero` en `edunn/initializers.py`, podemos ver que:
* Hereda de `Initializer`
* Implementa el método `initialize(self, p: np.ndarray)` que recibe un array de numpy para inicializar
* Utiliza `p[:]` para inicializar en 0 en lugar de `p = 0`. Hay dos razones importantes para esto:
    * Utilizar `p = 0` solo cambiaría la _variable local_ `p` en lugar de cambiar el _array de numpy_ al cual `p` apunta
    * Al utilizar `p[:]` estamos cambiando el __contenido__ del array de parámetros, que pertenece a la clase de la capa (en este caso `DummyLayer`)

Una vez creada la clase, podemos obtener el vector de parámetros `c` de la clase `AddValue`con el método `get_parameters()`


In [ ]:
# Crea una capa DummyLayer con 2 valores de entrada/salida
layer = DummyLayer(2, initializer=nn.initializers.Zero())
print(f"Nombre de la capa: {layer.name}")
print(f"Parámetros de la capa: {layer.get_parameters()}")
print()

# Por defecto, el inicializador ya es `Zero`
layer2 = DummyLayer(2)
print(f"Nombre de la capa: {layer2.name}")
print(f"Parámetros de la capa: {layer2.get_parameters()}")

# Acceso a los parámetros por nombre

El método `get_parameters()` devuelve un diccionario de parámetros, ya que admite tener más de un parámetro por capa.


Dado que ya sabemos en este caso cuál es el nombre del único parámetro de la capa, podemos acceder al mismo con su nombre en string, `'c'`:

In [ ]:
print(f"Parametros: {layer.get_parameters()}")
print(f"Parametro 'c': {layer.get_parameters()['c']}")


# Implementación de un Inicializador constante

Mientras que algunas veces los parámetros son inicializados a `0`, es muy común darles algún valor constante. 

Implementa el inicializador `Constant` que le asigna un valor o arreglo constante al parámetro, de modo que por ejemplo, se pueda inicializar `c` con todos valores `3` o con un vector de valores `[1, 2, 3, 4]`. 

Busca la clase `Constant` en el módulo `edunn/initializers.py` e implementa el método `initialize` para que permita inicializar el parámetro con un valor arbitrario.


In [ ]:
# Creamos una capa DummyLayer con 2 valores de salida (y también de entrada). 
# Los parámetros están todos inicializados con 3
valor = 3
layer = DummyLayer(2, initializer=nn.initializers.Constant(valor))

print(f"Nombre de la capa: {layer.name}")
print(f"Parámetro 'c' de la capa: {layer.get_parameters()['c']}")
utils.check_same(layer.get_parameters()['c'], np.array([3, 3]))
print()

# Creamos una capa DummyLayer con valores iniciales 1, 2, 3, 4
# Notar que estamos asegurando que la cantidad de valores del initializer Constant sea el mismo que el array valor
valor2 = np.array([1, 2, 3, 4])
layer2 = DummyLayer(4, initializer=nn.initializers.Constant(valor2))

print(f"Nombre de la capa 2: {layer2.name}")
print(f"Parámetro 'c' de la capa 2: {layer2.get_parameters()['c']}")

utils.check_same(layer2.get_parameters()['c'], valor2)